# Dude, Where's My House?
## Part 6: Using Model Predictions and Interacting with APIs
## Authors: Aman Hafez, Kavan Pandya, Yan Nusinovich
Notebook drafted by Kavan Pandya

### Imports

In [13]:
import os

# Import google_streetview for the api module
import googlemaps
import google_streetview.api
import google_streetview

import json

# Importing for reversing to the address
from pygeocoder import Geocoder

# importing zillow 
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults, GetUpdatedPropertyDetails

# import image meta data extractors
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

In [14]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
import pickle

In [15]:
# Get the api keys
from credentials_google import google_apikey
from zillowcreds_2 import zillow_apikey

### Functions

In [16]:
class ImageMetaData(object):
    '''
    Extract the metadata (exif data) from any image. 
    Returns GPS coordinates.
    '''
    
    exif_data = None
    image = None

    def __init__(self, img_path):
        self.image = Image.open(img_path)
        self.get_exif_data()


    def get_exif_data(self):
        """Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags"""
        exif_data = {}
        info = self.image._getexif()
        if info:
            for tag, value in info.items():
                decoded = TAGS.get(tag, tag)
                if decoded == "GPSInfo":
                    gps_data = {}
                    for t in value:
                        sub_decoded = GPSTAGS.get(t, t)
                        gps_data[sub_decoded] = value[t]

                    exif_data[decoded] = gps_data
                else:
                    exif_data[decoded] = value
        self.exif_data = exif_data
        return exif_data

    def get_if_exist(self, data, key):
        if key in data:
            return data[key]
        return None

    def convert_to_degress(self, value):

        """Helper function to convert the GPS coordinates 
        stored in the EXIF to degress in float format"""
        d0 = value[0][0]
        d1 = value[0][1]
        d = float(d0) / float(d1)

        m0 = value[1][0]
        m1 = value[1][1]
        m = float(m0) / float(m1)

        s0 = value[2][0]
        s1 = value[2][1]
        s = float(s0) / float(s1)

        return d + (m / 60.0) + (s / 3600.0)

    def get_lat_lng(self):
        """Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)"""
        lat = None
        lng = None
        exif_data = self.get_exif_data()
        #print(exif_data)
        if "GPSInfo" in exif_data:      
            gps_info = exif_data["GPSInfo"]
            gps_latitude = self.get_if_exist(gps_info, "GPSLatitude")
            gps_latitude_ref = self.get_if_exist(gps_info, 'GPSLatitudeRef')
            gps_longitude = self.get_if_exist(gps_info, 'GPSLongitude')
            gps_longitude_ref = self.get_if_exist(gps_info, 'GPSLongitudeRef')
            if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
                lat = self.convert_to_degress(gps_latitude)
                if gps_latitude_ref != "N":                     
                    lat = 0 - lat
                lng = self.convert_to_degress(gps_longitude)
                if gps_longitude_ref != "E":
                    lng = 0 - lng
                    
        return lat, lng

In [17]:
def reverse_lookup(lat, long, key=google_apikey):
    """Function to lookup addresses from latitude, longitude details using Google Maps API
    Args:
        lat (float): latitude as float
        long (float): longitude as float
        key (str): (default='YOURAPIKEYHERE') google maps api key
    Returns:
        returns a tuple with address (str), zipcode (str)
        """
    
    result = str(Geocoder(api_key=key).reverse_geocode(lat, long))
    location_details = result.split(",")
    address = location_details[0]
    zipcode = location_details[-2].strip().split(' ')[1]
    city = location_details[1]
    state = location_details[2].split(" ")[1]
    
    return address, zipcode, city, state

In [18]:
def get_zillow_estimate(key, address, zipcode):
    '''
    function to fetch estimated price of a house from its street address
    Args:
        key (str): zillow api key
        address (str): house street address
        zipcode (str): house zipcode
    Returns:
        amount (float): estimated house price from zillow
    '''
    
    zillow_data = ZillowWrapper(key)
    deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
    result = GetDeepSearchResults(deep_search_response)
    zestimate=0
    try:
        zestimate = float(result.zestimate_amount)
        
    except:
        pass
    
    if zestimate:
        amount = zestimate
        
    else:
        amount = 0.0
    
    return amount

In [19]:
# Define parameters for street view api

def google_streetviewer(location, key='YOURAPIKEY', heading=0, pitch=0):
    '''
    function to fetch google street view image from given address
    this image will serve as the reference 'before' image if house is damaged
    Args:
        location (str): address of the house
        key (str): google api key
        heading (int): vert angle of streetview image
        pitch (int): horz angle of streetview image
    Returns:
        image file : google streetview image of the given address 
    '''
    
    params = [{'size': '600x300', # max 640x640 pixels
               'location': location,
               'heading': heading,# might require adjusting
               'pitch': pitch, # might require adjusting
               'key': key
              }]

    # Create a results object
    results = google_streetview.api.results(params)

    # Download images to directory 'google-pics'
    return results.download_links('google-pics')

In [20]:
# check if user has image to upload
def user_input():
    """
    function that takes image from user to get estimated house price
    Returns:
        estimate (float): estimated house price
    """
    path_name = None
    check = input("Do you have an image of the front of your house to upload (y/n)? ")

    if check == 'y':
        path_name = input('\nEnter house image path on your computer, either from your home directory or from your current directory: ')
        # fetch location info from image
        meta_data =  ImageMetaData(path_name)
        latlng = meta_data.get_lat_lng()
        
        # if image contains location data
        if all(latlng):
            
            # fetch address from lat long
            addr = reverse_lookup(latlng[0], latlng[1], google_apikey)
            print('\n' + str(addr))

            # check if returned address is correct 
            response = input('\nIs this your address (y/n)? ')
            if response == 'n':
                # input address if returned address was incorrect
                print(('\nEnter your street address followed by a comma "," and then your zip code.'))
                new_addr = input('example: 100 Main St, 07110\n')
                new_addr = new_addr.split(',')
            else:
                new_addr = addr

            # fetch estimated amount from zillow
            estimate = get_zillow_estimate(zillow_apikey, new_addr, new_addr[1])
            print('\nThe original estimate from Zillow for your home\s value is: ${:,.2f}'.format(estimate))
            
            #get streetview image
            cond = input('\nDo you want a street view image of your house (y/n)? ')
            if cond == 'y':
                google_streetviewer(new_addr,google_apikey)

            return estimate, path_name
        
        # if image does not contain location data
        else:
            print('\nThis image does not have any location data.')
            print(('\nEnter your street address followed by a comma "," and then your zip code.'))
            new_addr = input('example: 100 Main St, 07110 \n')
            new_addr = new_addr.split(',')

            # fetch estimated amount from zillow
            estimate = get_zillow_estimate(zillow_apikey, new_addr, new_addr[1])
            print('\nThe original house value estimate from Zillow is: ${:,.2f}'.format(estimate))
        
            # get streetview image
            cond = input('\nDo you want a streetview image of your house (y/n)? ')
            if cond == 'y':
                google_streetviewer(new_addr,google_apikey)
        
            return estimate,path_name

    # if user has no image to upload:        
    else:
        print(('\nEnter your street address followed by a comma "," and then your zip code.'))
        new_addr = input('example: 100 Main St, 07110 \n')
        new_addr = new_addr.split(',')

        # fetch estimated amount from zillow
        estimate = get_zillow_estimate(zillow_apikey, new_addr, new_addr[1])
        print('\nThe original house estimate from Zillow is: ${:,.2f}'.format(estimate))
        
        # get streetview image
        cond = input('\nDo you want a streetview image of your house (y/n)? ')
        if cond == 'y':
            google_streetviewer(new_addr,google_apikey)
        
        return estimate,path_name


### Model Loading

In [21]:
model3 = Sequential()
model3.add(InputLayer(input_shape=(256, 256, 3)))

model3.add(Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same')) # add batch size and ES
model3.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model3.add(Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model3.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model3.add(BatchNormalization())
model3.add(Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model3.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model3.add(BatchNormalization())
model3.add(Flatten())
model3.add(Dense(units=100, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(units=100, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(units=3, activation='softmax'))

model3.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model3.load_weights("./model/final_model.h5")

In [22]:
def load_obj(name):
    with open('model/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
train_class_indices = load_obj("train_class_indices")

### Main

In [24]:
(estimate, path_name) = user_input()

Do you have an image of the front of your house to upload (y/n)? n

Enter your street address followed by a comma "," and then your zip code.
example: 100 Main St, 07110 
11810 Silent Valley Ln, 20878

The original house estimate from Zillow is: $612,098.0


Do you want a streetview image of your house (y/n)? y


In [29]:
# estimate the damage from model

# if user uploads photo:
if path_name:
    # run model with this image to get classification
    # ...
    direc = os.path.dirname(os.path.dirname(path_name))
    
    imagegen = ImageDataGenerator()
    test = imagegen.flow_from_directory(direc, class_mode=None, shuffle=False, batch_size=1, target_size=(256, 256))
    test.reset()
    pred = model3.predict_generator(test, verbose=0)
    
    predicted_class_indices=np.argmax(pred,axis=1)
    
    labels = (train_class_indices)
    labels = dict((v,k) for k,v in labels.items())
    predictions = [labels[k] for k in predicted_class_indices]
            
    if predictions[0] == 'Good':
        model_estimate = 0
    elif predictions[0] == 'Damaged':
        model_estimate = 0.5
    else:
        model_estimate = 1
    
    damage_estimate = estimate*model_estimate
    print('\nThe original value of your house was ${:,.2f}. The model estimates that your house is {}, and the estimated damage is ${:,.2f}'.format(estimate, predictions[0].lower(), damage_estimate))
    
# if user does not upload photo:
else:
    condition = input('\nEnter the condition of your house (Good, Damaged, Destroyed): ')
    
    if condition == 'Good':
        damage_estimate = 0
    elif condition == 'Damaged':
        damage_estimate = 0.5*estimate
    else:
        damage_estimate = estimate
        
    print('\nThe damage estimate is ${:,.2f}'.format(damage_estimate))


Enter the condition of your house (Good, Damaged, Destroyed): Damaged

The damage estimate is $306,049.00
